In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
from netCDF4 import Dataset
import os
import datetime

This data is via:</br>
https://www.climatologylab.org/wget-terraclimate.html

In [5]:
# specifying a subset of the data can make
# running iterations of functions much faster
CO_lat = [37, 41]
CO_long = [-109, -102]

NASA provides a guide for how to read in and manipulate netCDF4 data:</br>
https://www.star.nesdis.noaa.gov/atmospheric-composition-training/python_netcdf4.php

In [6]:
filepath = '../Data/simple_model/'
file_name = os.listdir(filepath)
len(file_name)

134

Each file contains monthly data for one year for one feature. There are 14 features in total, and I selected data for 2010 to the end of 2020, i.e. 11 years, so there should be 154 files here, but . . .

In [7]:
print(f"I was only able to download {len(file_name)} files.")

I was only able to download 134 files.


Running the scripts provided by the Climatology Lab returns a 302 error for many of the files meant to be downloaded. This is a temporary SSL error, meaning that if I run these scripts many times throughout the week, I'll be able to collect more and more files, as they become available at different times.

I needed to convert the day of the year (i.e. j in terms of datetime variables) into an actual date using the year I extracted from the file name. This StackOverflow post gives a concise explaination:
https://stackoverflow.com/questions/2427555/how-to-convert-year-and-day-of-year-to-date</br>

In [40]:
monthly_df = pd.DataFrame()
for i in range(len(file_name)):
    try:
        # complete filepath
        file = filepath + file_name[i]
        # creating netCDF4 object
        file_id = Dataset(file)
        # extracting year from file name
        year = int(file_name[i][-7:-3])
        # name of variable stored within file
        file_variable = list(file_id.variables.keys())[-1]
        # averaging the entire US's data for each day
        var = [file_id.variables[file_variable][c, :, :].data.mean() for c in range(0,12)]
        date = [datetime.datetime.strptime(f"{year} {f}", '%Y %j')\
                .strftime('%Y-%m-%d') for f in range(1, 13)]

        d = {'date': date, f"{file_variable}": var}
        temp_df = pd.DataFrame(data = d)
        monthly_df = pd.concat([monthly_df, temp_df], ignore_index=False)
        
    except:
        f"{file_id} not found!"
        
    monthly_df['date'] = pd.to_datetime(monthly_df['date'])
monthly_df = monthly_df.groupby('date').mean()

In [31]:
monthly_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 132 entries, 2010-01-01 to 2020-01-12
Data columns (total 12 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   burning_index_g                            108 non-null    float64
 1   potential_evapotranspiration               120 non-null    float64
 2   dead_fuel_moisture_1000hr                  108 non-null    float64
 3   dead_fuel_moisture_100hr                   96 non-null     float64
 4   precipitation_amount                       120 non-null    float64
 5   relative_humidity                          120 non-null    float64
 6   specific_humidity                          132 non-null    float64
 7   surface_downwelling_shortwave_flux_in_air  84 non-null     float64
 8   wind_from_direction                        72 non-null     float64
 9   air_temperature                            132 non-null    float64
 10  mean_va

In [32]:
monthly_df.head()

,burning_index_g,potential_evapotranspiration,dead_fuel_moisture_1000hr,dead_fuel_moisture_100hr,precipitation_amount,relative_humidity,specific_humidity,surface_downwelling_shortwave_flux_in_air,wind_from_direction,air_temperature,mean_vapor_pressure_deficit,wind_speed
date,,,,,,,,,,,,
2010-01-01,NaN,32767.0,32767.0,32767.0,32767.0,32767.0,32767.0,32767.0,NaN,32767.0,32767.0,32767.0
2010-01-02,NaN,32767.0,32767.0,32767.0,32767.0,32767.0,32767.0,32767.0,NaN,32767.0,32767.0,32767.0
2010-01-03,NaN,32767.0,32767.0,32767.0,32767.0,32767.0,32767.0,32767.0,NaN,32767.0,32767.0,32767.0
2010-01-04,NaN,32767.0,32767.0,32767.0,32767.0,32767.0,32767.0,32767.0,NaN,32767.0,32767.0,32767.0
2010-01-05,NaN,32767.0,32767.0,32767.0,32767.0,32767.0,32767.0,32767.0,NaN,32767.0,32767.0,32767.0


In [33]:
monthly_df.tail()

,burning_index_g,potential_evapotranspiration,dead_fuel_moisture_1000hr,dead_fuel_moisture_100hr,precipitation_amount,relative_humidity,specific_humidity,surface_downwelling_shortwave_flux_in_air,wind_from_direction,air_temperature,mean_vapor_pressure_deficit,wind_speed
date,,,,,,,,,,,,
2020-01-08,32767.0,32767.0,32767.0,32767.0,32767.0,32767.0,32767.0,NaN,NaN,32767.0,NaN,NaN
2020-01-09,32767.0,32767.0,32767.0,32767.0,32767.0,32767.0,32767.0,NaN,NaN,32767.0,NaN,NaN
2020-01-10,32767.0,32767.0,32767.0,32767.0,32767.0,32767.0,32767.0,NaN,NaN,32767.0,NaN,NaN
2020-01-11,32767.0,32767.0,32767.0,32767.0,32767.0,32767.0,32767.0,NaN,NaN,32767.0,NaN,NaN
2020-01-12,32767.0,32767.0,32767.0,32767.0,32767.0,32767.0,32767.0,NaN,NaN,32767.0,NaN,NaN


In [38]:
monthly_df.to_csv(r'../Data/output_csv_files/monthly_df.csv', index_label = False)

I've learned that it's extremely important to include `parse_dates = True` in order for the dataframe to remain a datetime index. Not doing this will result in a standard index being applied with no warning, which can be annoying.

In [39]:
pd.read_csv(r'../Data/output_csv_files/monthly_df.csv', parse_dates = True).info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 132 entries, 2010-01-01 to 2020-01-12
Data columns (total 12 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   burning_index_g                            108 non-null    float64
 1   potential_evapotranspiration               120 non-null    float64
 2   dead_fuel_moisture_1000hr                  108 non-null    float64
 3   dead_fuel_moisture_100hr                   96 non-null     float64
 4   precipitation_amount                       120 non-null    float64
 5   relative_humidity                          120 non-null    float64
 6   specific_humidity                          132 non-null    float64
 7   surface_downwelling_shortwave_flux_in_air  84 non-null     float64
 8   wind_from_direction                        72 non-null     float64
 9   air_temperature                            132 non-null    float64
 10  mean_va

---------

Unsuccessful daily model:

The Climatology Lab also had a very interesting dataset containing daily satelite data with a wide array of variables like NDVI (which could help identify peak greenness), PDSI (Palmer Drought Severity Index), and most of the same variables found in the monthly dataset. Unfortunately, the same issue occured with the monthly dataset, i.e. many files were not able to download. In addition, since this is daily satelite data, the amount of data required to make a model going back to the year 1980 ended up being about 30 Gigabytes. Due to the amount of missing data and the difficulty of reliably imputing daily climate data for entire years, this model did not end up being feasable. I originally downloaded, processed and examined data from this source going back to 1990, but the sheer amount of data and the fact that so much was missing led me to delete most of it from my local machine. Here is the work I was able to accomplish, only including data from 2010 to 2021 for the purpose of demonstration.

I grouped the data by year in order to examine how many files are missing (each file contains data for one year for one feature)

Data via https://www.climatologylab.org/wget-gridmet.html

In [ ]:
df = pd.DataFrame()

filepath = '../Data/gridmet/'
file_name = os.listdir(filepath)
len(file_name)

year_df = pd.DataFrame()
for i in range(len(file_name)):
    # complete filepath
    file = filepath + file_name[i]
    # creating netCDF4 object
    file_id = Dataset(file)
    # extracting year from file name
    year = int(file_name[i][-7:-3])
    # name of variable stored within file
    file_variable = list(file_id.variables.keys())[-1]
    # averaging the entire US's data for each day
    var = [file_id.variables[file_variable][:, :, c].data.mean() for c in range(0,365)]
    date = [datetime.datetime.strptime(f"{year} {f}", '%Y %j')\
            .strftime('%Y') for f in range(1, 366)]

    d = {'date': date, f"{file_variable}": var}
    temp_df = pd.DataFrame(data = d)

    df = pd.concat([df, temp_df], ignore_index=False)
year_df = df.groupby('date').mean()

year_df.tail()

The OSError is simply letting me know that the `wget` script I used to download the data is still in the data folder, so there's no need for concern.

In [ ]:
year_df.head()

In [ ]:
sns.lineplot(year_df['aet'])